<a href="https://colab.research.google.com/github/Lalitha1704/Football_Video_Summarization/blob/main/Copy_of_sports_video_sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics opencv-python moviepy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.5 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

VIDEO_PATH = list(uploaded.keys())[0]
print("Input video:", VIDEO_PATH)

Saving 2.mp4 to 2.mp4
Input video: 2.mp4


In [ ]:
import cv2
from ultralytics import YOLO
from moviepy.editor import VideoFileClip, concatenate_videoclips

model = YOLO("yolov8l.pt")  # lightweight model

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [ ]:
cap = cv2.VideoCapture(VIDEO_PATH)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

important_times = []
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=0.4, verbose=False)

    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])

            # COCO class 32 = sports ball
            if cls == 32:
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Assume goal area is right side
                if x2 > frame_width * 0.7:
                    time_sec = frame_count // fps
                    important_times.append(time_sec)

    frame_count += 1

cap.release()

important_times = sorted(set(important_times))
print("Raw detected times:", important_times)

Raw detected times: [10, 11, 12, 14, 18, 20, 21]


In [ ]:
if len(important_times) == 0:
    print("No goal attempts detected")
    exit()

GROUP_GAP = 5  # seconds

grouped_times = []
current_group = [important_times[0]]

for t in important_times[1:]:
    if t - current_group[-1] <= GROUP_GAP:
        current_group.append(t)
    else:
        grouped_times.append(current_group)
        current_group = [t]

grouped_times.append(current_group)

event_times = [group[len(group)//2] for group in grouped_times]
print("Final goal attempts:", event_times)

Final goal attempts: [14]


In [ ]:
POST_EVENT_CHECK = 3      # seconds
GOAL_REGION_RATIO = 0.85 # deeper goal area
MIN_FRAMES_IN_GOAL = 5   # persistence threshold


def is_goal(video_path, event_time, fps, frame_width):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, int(event_time * fps))

    frames_to_check = int(4 * fps)

    positions = []
    detected_frames = 0

    for _ in range(frames_to_check):
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, conf=0.4, verbose=False)

        for r in results:
            for box in r.boxes:
                if int(box.cls[0]) == 32:  # sports ball
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cx = (x1 + x2) // 2
                    positions.append(cx)
                    detected_frames += 1

    cap.release()

    # -------- HEURISTICS --------

    if detected_frames == 0:
        return False  # No info → MISS

    # 1️⃣ Deep goal region presence
    deep_goal = any(x > frame_width * 0.85 for x in positions)

    # 2️⃣ Sudden slowdown (ball stops)
    if len(positions) >= 3:
        speeds = [abs(positions[i+1] - positions[i]) for i in range(len(positions)-1)]
        slowdown = min(speeds) < 3
    else:
        slowdown = False

    # 3️⃣ Ball disappears early
    disappearance = detected_frames < frames_to_check * 0.3

    # Decision rule (any 2 signals)
    score = sum([deep_goal, slowdown, disappearance])

    return score >= 2


In [ ]:
event_results = []  # (time, label)

for t in event_times:
    label = "GOAL" if is_goal(VIDEO_PATH, t, fps, frame_width) else "MISS"
    event_results.append((t, label))

print("\nEvent classification:")
for t, label in event_results:
    print(f"Time {t}s → {label}")


Event classification:
Time 14s → MISS


In [ ]:
BUFFER_BEFORE = 3
BUFFER_AFTER = 3

segments = []  # (start, end, label)
for t, label in event_results:
    start = max(0, t - BUFFER_BEFORE)
    end = t + BUFFER_AFTER
    segments.append((start, end, label))

In [ ]:
merged_segments = []

for start, end, label in sorted(segments):
    if not merged_segments:
        merged_segments.append([start, end, label])
    else:
        last_start, last_end, last_label = merged_segments[-1]
        if start <= last_end:
            merged_segments[-1][1] = max(last_end, end)
        else:
            merged_segments.append([start, end, label])

print("\nFinal video segments:")
for seg in merged_segments:
    print(seg)


Final video segments:
[11, 17, 'MISS']


In [ ]:
video = VideoFileClip(VIDEO_PATH)
clips = []

for start, end, label in merged_segments:
    clips.append(video.subclip(start, end))

final_video = concatenate_videoclips(clips)
final_video.write_videofile(
    "goal_summary_final.mp4",
    codec="libx264",
    audio_codec="aac"
)

Moviepy - Building video goal_summary_final.mp4.
MoviePy - Writing audio in goal_summary_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video goal_summary_final.mp4



Moviepy - Done !
Moviepy - video ready goal_summary_final.mp4


In [ ]:
from google.colab import files
files.download("goal_summary_final.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>